Fill in the missing code (#####). Here you learn to use the spearman_metric function. Check the very important question at the end.

IMPORTANT:<br>
We have argued that the Spearman Correlation and the Profit Factor are good non-linear losses or metrics <br>
to measure the performance of many financial models, especially regression models. <br>

Both TensorFlow/Keras and Scikit-Learn allow custom metrics (scoring functions). <br>
Only TensorFlow/Keras allows custom losses (cost functions). <br>

TensorFlow/Keras custom metrics are relatively easy to program. <br>
TensorFlow/Keras custom losses are hard: they need to be differentiable. <br>

In this notebook, <br>
we have programmed a custom spearman_correlation_loss, which <br>
is differentiable but an rough approximation to the real spearman correlation.<br>
Unfortunately, <br>
the neural network cannot learn well with this custom spearman_correlation_loss (you can try this below). <br>
So, for now, if you want to use the Spearman Correlation at all with TensorFlow/Keras, <br>
you can only use it as a metric and not as a loss.<br>
This may change in the future. <br>

In this notebook you will learn to use the combination of loss=mean squared error and metric= spearman correlation.<br>
In fact, the combination of loss=mean squared error and metric=any list of metrics is possible.<br>
    
    
It is also possible to use the combination loss=pearson correlation and metric=any list of metrics. <br>
See: 5.pearson_corr_incomplete.ipynb

In [15]:
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import numpy as np
# from keras.layers import Dense
# from keras.models import Sequential
# from keras.optimizers import SGD
from matplotlib import pyplot
from scipy.stats import spearmanr
from scipy.stats import pearsonr
# import keras.backend as K
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
# import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K


Here you see how to calculate the spearman correlation using numpy (1a), tensors (1b) and scipy.stats.spearmanr (3).
The results are similar. The comparison is in 3.
See: https://archive.md/VfNkG

In [16]:
#1 spearman from scratch using numpy
def spearman_correlation(predictions, targets):
    if not isinstance(predictions, pd.Series):
        predictions = pd.Series(predictions)
    ranked_preds = predictions.rank(pct = True, method = "first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

#2 spearman from scratch using tensors
def corrcoef(x, y):
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(y)
    xm, ym = x - mx, y - my
    r_num = tf.math.reduce_sum(xm * ym)
    r_den = tf.norm(xm) * tf.norm(ym)
    return r_num / (r_den + tf.keras.backend.epsilon())

#3 spearman using tensors
def tf_spearman_correlation(predictions, targets):
    ranked_preds = tf.cast(tf.argsort(tf.argsort(predictions, stable = True)), targets.dtype)
    return corrcoef(ranked_preds, targets)

Next we use tensors to program a spearman metric. Note the use of py_func (a shortcut).

py_func is a tf wrapper for a python function. py_func returns a tensor.
Below we use py_func to wrap around the python function spearmanr.
This use of py_func works in my setup but it does not always work.
If you have problems with it, 
just use the spearman_metric underneath (commented out) that uses tensors.

In [ ]:
#4 spearman using scipy
def spearman_metric(y_true, y_pred):
    """Spearman correlation coefficient using a tf wrapper for a python function"""
    r = tf.py_function(spearmanr, inp=[y_true, y_pred], Tout=tf.float32)
    return  r

Next we use tensors to program a spearman loss that is compatible with Keras and TensorFlow. Note the argsort operation, which is not differentiable, cannot be used, so we use soft_rank instead. soft_rank gives approximate results.

In [ ]:
#5 spearman using soft_rank
def soft_rank(x, epsilon=1e-6):
    pairwise_differences = x[:, None] - x[None, :]
    abs_diff = tf.abs(pairwise_differences)
    soft_rank = tf.reduce_sum(1 / (1 + abs_diff / epsilon), axis=1)
    return soft_rank


def spearman_correlation_loss(y_true, y_pred):
    y_true_rank = soft_rank(y_true)
    y_pred_rank = soft_rank(y_pred)

    mean_y_true_rank = tf.reduce_mean(y_true_rank)
    mean_y_pred_rank = tf.reduce_mean(y_pred_rank)

    covariance = tf.reduce_mean((y_true_rank - mean_y_true_rank) * (y_pred_rank - mean_y_pred_rank))
    std_y_true_rank = tf.math.reduce_std(y_true_rank)
    std_y_pred_rank = tf.math.reduce_std(y_pred_rank)

    epsilon = 1e-6
    spearman_corr = covariance / (std_y_true_rank * std_y_pred_rank + epsilon)
    scaling_factor = 100000 
    return -spearman_corr*scaling_factor
#note: A neural network training process minimizes the loss, so a negative Spearman correlation loss ensures that maximizing the correlation reduces the loss.

In [24]:
# Use of these functions
targets = np.array([0.0, 0.25, 0.5, 0.75, 1.0], dtype = np.float32)
predictions = np.random.rand(targets.shape[0])

print("Spearman using numpy:", spearman_correlation(predictions, targets))

# result1 = tf_spearman_correlation(tf.convert_to_tensor(predictions, dtype=tf.float32), tf.convert_to_tensor(targets, dtype=tf.float32))
# with tf.compat.v1.Session() as sess:
#     print("Spearman using tensors:", sess.run(result1))

result1 = tf_spearman_correlation(
    tf.convert_to_tensor(predictions, dtype=tf.float32),
    tf.convert_to_tensor(targets, dtype=tf.float32)
)

print("Spearman using tensors:", result1.numpy())

# Spearman using scipy
# result2 = spearman_metric(targets, predictions)
# with tf.compat.v1.Session() as sess:
#     print("Spearman using scipy:", sess.run(result2))

result2 = spearman_metric(
    tf.convert_to_tensor(targets, dtype=tf.float32),
    tf.convert_to_tensor(predictions, dtype=tf.float32)
)

print("Spearman using scipy:", result2.numpy())


# Spearman using soft_rank
# result3 = spearman_correlation_loss(tf.convert_to_tensor(targets, dtype=tf.float32), tf.convert_to_tensor(predictions, dtype=tf.float32))
# with tf.compat.v1.Session() as sess:
#     print("Spearman using soft_rank:", -sess.run(result3)) #multiply by negative to undo the negative sign in the loss function
result3 = spearman_correlation_loss(
    tf.convert_to_tensor(targets, dtype=tf.float32),
    tf.convert_to_tensor(predictions, dtype=tf.float32)
)

print("Spearman using soft_rank:", -result3.numpy())  # negate to convert loss to positive correlation


Spearman using numpy: -0.09999999999999996
Spearman using tensors: -0.1
Spearman using scipy: -0.1
Spearman using soft_rank: 3.8585856


In [25]:
num_features = 20
# generate regression dataset
X, y = make_regression(n_samples=1000, n_features=num_features, noise=0.1, random_state=1)

# split into train and test
n_train = 500
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# reshape 1d arrays to 2d arrays
trainy = trainy.reshape(len(trainy), 1)
testy = testy.reshape(len(trainy), 1)

# create scaler
scaler = StandardScaler()
# fit scaler on training dataset
scaler.fit(trainy)
# transform training dataset
trainy = scaler.transform(trainy)
# transform test dataset
testy = scaler.transform(testy)

# fit scaler on training dataset
scaler.fit(trainX)
# transform training dataset
trainX = scaler.transform(trainX)
# transform test dataset
testX = scaler.transform(testX)

In [26]:
# mlp with scaled outputs on the regression problem with custom loss and custom metric
# define model
model = Sequential()
model.add(Dense(25, input_dim=20, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='linear'))
# compile model using as loss: 'mean_squared_error', and as metric: spearman_metric
model.compile(loss=spearman_correlation_loss,
              optimizer=SGD(learning_rate=0.01, momentum=0.9),
              metrics=[spearman_metric]) 
#the model won't learn well if spearman_correlation_loss is used, but try it and see the results.
    
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=100, verbose=1)
# evaluate the model
train_e = model.evaluate(trainX, trainy, verbose=1)
test_e = model.evaluate(testX, testy, verbose=1)
print('Train loss: %.3f, Test loss: %.3f' % (train_e[0], test_e[0])) #when using custom loss and custom metric
print('Train metric: %.3f, Test metric: %.3f' % (train_e[1], test_e[1])) #when using custom loss and custom metric
#plot loss during training
pyplot.title('Loss / Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Epoch 1/100


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Cannot take the length of shape with unknown rank.

Does Keras use metric functions (including custom metric functions) for anything other than reporting?
You can use a metric function in a callback to make Keras stop training when the metric function's score
is no longer improving.
See:
https://archive.md/OLvkZ
https://archive.md/VTS87
https://archive.md/RV8A8